In [2]:
import pandas as pd
import numpy as np
import glob
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Okt
import re
import pickle
import json

In [7]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DF=pd.read_csv(r"C:\Users\kdp\Desktop\KDW\EX_FLASK\project\data\건강관련톡.csv",encoding='utf-8-sig', header=None)

DF.head()

,0,1
0,건강,그래 건강이 좀 안 좋아지면 비뇨기과도 다니면 되는 거지...
1,건강,뭐든 서양 약들로 처방 받으면 건강에 더 안 좋아...
2,건강,하하 *이 남성 쪽으로 건강이 안 좋나 보네
3,건강,*는 뭔데 자신감 넘치네?
4,건강,나는 쟤한테 물어보면 될 거 아냐


In [8]:
# 텍스트 데이터와 라벨 추출
texts = DF[1].tolist()
labels = DF[0].tolist()


In [9]:
# 형태소 분석기 객체 생성
okt = Okt()

def make_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return set(stopwords)

# 불용어 파일 경로 설정
stopwords_path = r"C:\Users\kdp\Desktop\KDW\EX_FLASK\project\data\stopwords.txt"
stopwords = make_stopwords(stopwords_path)

# 추가 불용어 목록
additional_stopwords = {'키키', '아', '나', '는', '그래', '응', '응응', '휴', '어휴', '에고', '야', '진짜', '엉', '웅', '하하', '너', '가', '든', 
                        '앜', '헐', '어', '여기', '에', '다', '아니', '근데', '악', '히익', '챡챡챡챡', '으', '도', '네', '넵', '오', '그냥', 
                        '그럼', '그거', '그러네','그러게', '헐', '엥', '음', '을', '이', '와', '역시', '어어', '그래도', '맞아', '그', '막', 
                        '그렇지', '넹', '어머', '하', '우와', '오호', '오오','와우'}

# 불용어 목록에 추가 불용어를 결합
stopwords.update(additional_stopwords)

# 파일 경로 설정
file_path = r"C:\Users\kdp\Desktop\KDW\EX_FLASK\project\data\건강관련톡.csv"
file = file_path

# 데이터프레임 불러오기
DF = pd.read_csv(file_path, encoding='utf-8-sig')

# 데이터 저장 리스트
cleaned_texts = []

# 데이터프레임 DF에서 텍스트 데이터 전처리
for text in DF.iloc[:, 1].dropna():  # 텍스트 컬럼을 처리
    # 특수 문자 및 기호 제거 (한글, 공백만)
    text = re.sub(r'[^\가-힣\s]', '', text)
    
    # 형태소 분석으로 단어 분리
    words = okt.morphs(text)
    
    # 감탄사, 단독 자음/모음이 포함된 단어와 불용어 제거
    filtered_words = [
        word for word in words 
        if re.fullmatch(r'[가-힣]+', word) and word not in stopwords  # 한글로 이루어진 단어만 허용, 불용어 제거
    ]
    
    # 단어들을 다시 결합하여 전처리된 텍스트 저장
    cleaned_comment = ' '.join(filtered_words)
    cleaned_texts.append(cleaned_comment)

# 기존 데이터프레임에 수정된 텍스트 추가
DF.iloc[:, 1] = cleaned_texts + [''] * (len(DF) - len(cleaned_texts))  # 원래 길이와 맞추기

# 빈 문자열만 남은 행 제거
df_filtered = DF[DF.iloc[:, 1] != '']

# # 수정된 데이터프레임을 저장
# output_path = os.path.splitext(file)[0] + '_1.csv'
# df_filtered.to_csv(output_path, index=False, header=False)

# print(f"{output_path} 파일로 저장 완료")
# print("모든 파일 처리가 완료되었습니다.")


In [10]:
vocab = {}  # 통합된 어휘 사전

# 기존 단어 사전 불러오기
if os.path.exists("vocab.json"):
    with open("vocab.json", "r") as f:
        vocab = json.load(f)
else:
    vocab = {}  # 파일이 없으면 빈 vocab 생성

# 각 파일에서 어휘 사전 추가
for sentence in cleaned_texts:
    if isinstance(sentence, float):
        continue
    
    words = sentence.split()
    for word in words:
        if word not in vocab:
            vocab[word] = len(vocab) + 1  # 새로운 단어를 vocab에 추가하고 인덱스 부여

vocab_size = len(vocab) + 1
print("Vocabulary size:", vocab_size)

# 단어 사전 JSON 파일로 저장 (덮어쓰기 아님, 기존 단어 사전에 누적 저장)
with open("vocab.json", "w") as f:
    json.dump(vocab, f, ensure_ascii=False, indent=4)


Vocabulary size: 33172


In [11]:
vocab

{'뭐': 1,
 '서양': 2,
 '약': 3,
 '들': 4,
 '로': 5,
 '처방': 6,
 '받으면': 7,
 '건강': 8,
 '더': 9,
 '안': 10,
 '좋아': 11,
 '남성': 12,
 '쪽': 13,
 '으로': 14,
 '좋나': 15,
 '보네': 16,
 '뭔': 17,
 '데': 18,
 '자신감': 19,
 '넘치네': 20,
 '쟤': 21,
 '한테': 22,
 '물어보면': 23,
 '될': 24,
 '거': 25,
 '아냐': 26,
 '난': 27,
 '탈모': 28,
 '좀': 29,
 '먹어야': 30,
 '같아': 31,
 '이는': 32,
 '먹어서': 33,
 '나아도': 34,
 '괜찮을': 35,
 '듯': 36,
 '자꾸나': 37,
 '머리털': 38,
 '뭐라': 39,
 '하지': 40,
 '마라': 41,
 '스트레스': 42,
 '받지': 43,
 '마': 44,
 '만': 45,
 '병': 46,
 '의': 47,
 '근원': 48,
 '그러니까': 49,
 '보고': 50,
 '운동': 51,
 '라고': 52,
 '주지': 53,
 '받으면서': 54,
 '하면': 55,
 '풀려고': 56,
 '해서': 57,
 '잘': 58,
 '된대': 59,
 '그게': 60,
 '맞냐': 61,
 '이중': 62,
 '잖아': 63,
 '그건': 64,
 '너무': 65,
 '변태': 66,
 '적': 67,
 '인': 68,
 '성향': 69,
 '이야': 70,
 '편안하고': 71,
 '싶어': 72,
 '적당한': 73,
 '은': 74,
 '좋다고': 75,
 '하다못해': 76,
 '길거리': 77,
 '산책': 78,
 '하는': 79,
 '것': 80,
 '두': 81,
 '시간': 82,
 '씩': 83,
 '하는데': 84,
 '괜찮아': 85,
 '좋다': 86,
 '못': 87,
 '해': 88,
 '팔딱': 89,
 '거리': 90,
 '본데': 91,
 '나도': 92